In [2]:
# from data.data_loader import *
# from model import *
# import time
# from utils_lr import *
import torch

In [12]:
class RNN_depth_trainer:
    def initDataloader(self, dataset_dir, batch_size, img_height, img_widht, num_views, num_epochs, is_training):
        
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.num_views = num_views
        self.num_epochs = num_epochs
        
        initloader=DataLoader(dataset_dir, batch_size, img_height, img_width, num_epochs, self.num_views)
        
        dataLoader=initloader.inputs(is_training)
        
        return dataLoader
    
    def load_data(self, dataLoader):
        dataiter = iter(dataLoader)  #this could be an issue
        data_dict = dataiter.next()

        return data_dict
    
    def construct_model(self, data_dict):
    #Forward    
        image_seq=data_dict['image_seq']
        hidden_state = [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
        hidden_state_pos = [None, None, None, None, None, None, None]
        est_poses = []
        
        for i in range(self.num_views):
            
            image=image_seq[0:-1, 0:-1, self.img_width*i: int(self.img_width), 0:-1]
            image=torch.reshape(image,([self.batch_size, self.image_height,self.img_widht,3]))
            
            pred_depth, hidden_state = rnn_depth_net_encoderlstm()
            pred_pose, hidden_state_pose = pose_net(torch.cat((image, pred_depth),3), hidden_state_pose, is_training=True)
            est_poses.append(pred_pose)
            
            if i==0:
                est_depths=pred_depth
                
            else:
                est_depths = torch.cat((est_depths, pred_depths),2)
                
    #Backward
        
        image_seq=data_dict['image_seq']
        hidden_state = [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]
        hidden_state_pose= [None,None,None,None,None,None,None]
        est_poses_bw = []
        
        for i in range(self.num_views-1, -1, -1):
            
            image=image_seq[0:-1, 0:-1, self.img_width*i:int(self.img_width), 0:-1]
            depth=data_dict['depth_seq'][0:-1, 0:-1, self.img_widht*i:int(self.img_width), 0:-1]
            
            image=torch.reshape(image, [self.batch_size, self.img_height, self.img_width,3])
            
            pred_depth, hidden_state = rnn_depth_net_encoderlstm(image, hidden_state,is_training=True)
            pred_pose, hidden_state_pose = pose_net(torch.concat((image,pred_depth),3), hidden_state_pose,is_training=True)
            
            est_poses_bw.append(pred_pose)
            
            if i==self.num_views-1:
                est_depths_bw=pred_depth
                depth_seq_bw=depth
            else:
                est_depth_bw = torch.cat((est_depths_bw, pred_depth), 2)
                depth_seq_bw = torch.cat((depth_seq_bw, depth),2)
                
        data_dict['depth_seq_bw'] = depth_seq_bw
        
        return [est_depths, est_poses, est_depths_bw, est_poses_bw]
    
    
    def compute_losses(self, estimates, data_dict, global_step):
        est_depths = estimates[0]
        est_poses = estimates[1]
        est_depths_bw=estimates[2]
        est_depths_bw=estimates[3]
        
        all_losses = []
        output_dict = {}
        
        diff = torch.abs(label-pred)
        diff = torch.where(torch.isinf(diff), torch.zeros_like(diff), diff)
        diff = torch.where(torch.isnan(diff), torch.zeros_like(diff), diff)
        dif  = torch.count_nonzero(diff).float32
        
        if v_weight is not None:
            diff = torch.mul(diff, v_weight) #Elemetn Wise Product
            
        if v_weight is not None:
            return torch.sum(diff)/torch.count_nonzero(v_weight).float32
        
        else:
            return torch.sum(diff)/div
        
        def smooth_l1_loss(y_true, y_pred, v_weight = None):
            """Implements Smooth L1 Loss
            y_true and y_pred are typically: [N, 4], but could be any shape.
            """
            diff = torch.abs(y_true - y_pred)
            diff = torch.where(torch.isinf(diff), torch.zeros_like(diff), diff)
            diff = torch.where(torch.isnan(diff), torch.zeros_like(diff), diff)

            if v_wieght is not None:
                diff = torch.mul(diff, v_weight)
            less_than_one = torch.gt(diff,1.0).float32
            loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)

            if v_weight is not None:
                return torch.sum(loss)/torch.count_nonzero(v_weight).float32
            else:
                return torch.mean(loss)

        def gradient(pred, delta):
            D_dx = pred[:, delta:, :, :] - pred[:, :-delta, :, :]
            D_dy = pred[:, :, delta:, :] - pred[:, :, :-delta, :]

            return D_dx, D_dy

        def compute_smooth_loss(img, disp, delta):
            disp_gradients_x, disp_gradients_y = gradient(disp, delta)
            image_gradients_x, image_gradients_y = gradient(img, delta)

            weights_x = torch.exp(-torch.mean(torch.abs(image_gradients_x), 3))
            weights_y = torch.exp(-torch.mean(torch.abs(image_gradients_y), 3))

            smoothness_x = disp_gradients_x * weights_x
            smoothness_y = disp_gradients_y * weights_y

            return torch.mean(torch.abs(smoothness_x)) + torch.mean(torch.abs(smoothness_y))

        def SSIM(x,y):
            C1 = 0.01 ** 2
            C2 = 0.03 ** 2

            mu_x = F.avg_pool2d(x, 3, 1, 'SAME')
            mu_y = F.avg_pool2d(y, 3, 1, 'SAME')

            sigma_x = F.avg_pool2d(x ** 2, 3,1 , 'SAME') - mu_x ** 2
            sigma_y = F.avg_pool2d(y ** 2, 3,1 , 'SAME') - mu_y ** 2
            sigma_xy = F.avg_pool2d(x * y , 3, 1, 'SAME') - mu_x * mu_y

            SSIM_n = (2 * mu_x * mu_y + C1) * (2 * sigma_xy + C2)
            SSIM_d = (mu_x ** 2 + mu_y ** 2 + C1) * (sigma_x + sigma_y + C2)

            SSIM = SSIM_n / SSIM_d

            return torch.clamp((1-SSIM)/2, 0, 1)

        #Write This Function
        #def image_similarity(x, y, v_weight = None, alpha=1.0):
        #ss

        # Direct inverse depth loss
        depth_loss_fw = torch.zeros([])
        gt_depth_fw = 1/data_dict['depth_seq']
        depth_loss_fw = l1loss(est_depths, gt_depth_fw)
        all_losses.append(depth_loss_fw*10)

        output_dict['depth'] = est_depths

        #Backward depth
        depth_loss_bw = torch.zeros([]).float32
        gt_depth_bw = 1.0/data_dict['depth_seq_bw']
        depth_loss_bw = l1loss(est_depths_bw, gt_depth_bw)
        all_losses.append(depth_loss_bw*10)

        output_dict['depth_bw'] = est_depths_bw

        #Gradient Loss

        gradient_loss_fw=torch.zeros([]).float32

        for i in range(self.num_views):
            depth_slice = est_depths[0:-1, 0:-1, (self.img_width*i):int(self.img_width), 0:-1]

            gt_depth_slice = gt_depth_fw[0:-1, 0:-1, self.img_width*i:int(self.img_width), 0:-1]

            image_slice = data_dict['image_seq'][0:-1, 0:-1, self.img_width*i:int(self.img_width), 0:-1]

            Dx1, Dy1 = gradient(depth_slice,1); gtDx1, gtDy1 = gradient(gt_depth_slice,1)
            Dx2, Dy2 = gradient(depth_slice,2); gtDx2, gtDy2 = gradient(gt_depth_slice,2)
            Dx4, Dy4 = gradient(depth_slice,4); gtDx4, gtDy4 = gradient(gt_depth_slice,4)
            Dx8, Dy8 = gradient(depth_slice,8); gtDx8, gtDy8 = gradient(gt_depth_slice,8)

            gradient_loss_fw += (l1loss(Dx1, gtDx1)+
                               l1loss(Dy1, gtDy1))
            gradient_loss_fw += (l1loss(Dx2, gtDx2)+
                           l1loss(Dy2, gtDy2))
            gradient_loss_fw += (l1loss(Dx4, gtDx4)+
                           l1loss(Dy4, gtDy4))
            gradient_loss_fw += (l1loss(Dx8, gtDx8)+
                           l1loss(Dy8, gtDy8))

        all_losses.append(gradient_loss_fw*20)

        #Backward Gradient
        gradient_loss_bw = torch.zeros([]).float32

        for i in range(self.num_views):

            depth_slice=est_depths_bw[0:-1, 0:-1, self.img_width*i:int(img_width), 0:-1]
            gt_depth_slice = gt_depth_bw[0:-1, 0:-1, self.img_width*i:int(img_width), 0:-1]
            image_slice = data_dict['image_seq'][0:-1, 0:-1, self.img_width*(self.num_views-i-1):int(self.img_width), 0:-1]

            Dx1, Dy1 = gradient(depth_slice,1); gtDx1, gtDy1 = gradient(gt_depth_slice,1)
            Dx2, Dy2 = gradient(depth_slice,2); gtDx2, gtDy2 = gradient(gt_depth_slice,2)
            Dx4, Dy4 = gradient(depth_slice,4); gtDx4, gtDy4 = gradient(gt_depth_slice,4)
            Dx8, Dy8 = gradient(depth_slice,8); gtDx8, gtDy8 = gradient(gt_depth_slice,8)

            gradient_loss_bw += (l1loss(Dx1, gtDx1)+
                               l1loss(Dy1, gtDy1))
            gradient_loss_bw += (l1loss(Dx2, gtDx2)+
                           l1loss(Dy2, gtDy2))
            gradient_loss_bw += (l1loss(Dx4, gtDx4)+
                           l1loss(Dy4, gtDy4))
            gradient_loss_bw += (l1loss(Dx8, gtDx8)+
                               l1loss(Dy8, gtDy8))

        all_losses.append(gradient_loss_bw*20)

        #Pose Loss-Image reconstruction
        pose_loss_fw = torch.zeros([]).float32
        mask_loss_fw = torch.zeros([]).float32


        mask_regular = torch.ones((self.batch_size, self.img_height, self.img_width, 1))

        #Backward pose
        pose_loss_bw = torch.zeros([]).float32
        mask_loss_bw = torch.zeros([]).float32


        consist_loss_fw = torch.zeros([]).float32
        consist_loss_bw = torch.zeros([]).float32

        # Multi-view image reprojection loss
        for i in range(self.num_views):

            if i==0:

                depth_slice_fw = est_depths[0:-1, 0:-1, self.img_width*i:int(self.img_width), 0:-1]

                depth_slice_bw = est_depths_bw[0:-1, 0:-1, self.img_width*i:int(self.img_width), 0:-1]

                image_slice_fw = data_dict['image_seq'][0:-1, 0:-1, self.img_width*i: int(self.img_width), 0:-1]

                image_slice_bw = data_dict['image_seq'][0:-1, 0:-1, self.img_width*(self.num_views-1): int(self.img_width), 0:-1]

                depth_slice_fw.reshape([self.batch_size, self.img_height, self.img_width, 1])
                depth_slice_bw.reshape([self.batch_size, self.img_height, self.img_width, 1])
                image_slice_fw.reshape([self.batch_size, self.img_height, self.img_width, 3])
                image_slice_bw.reshape([self.batch_size, self.img_height, self.img_width, 3])

                proj_img, wmask, flow_fw = projective_inverse_warp(image_slice_fw, torch.squeeze( 1.0/depth_slice_fw, axis=3), est_poses[i], data_dict['intrinsics'], format='eular')

                pose_loss_fw += image_similarity(image_slice_fw, proj_img, wmask)*0.1 #Shouldnt work
                mask_loss_fw +=l1loss(mask_regular, wmask)*0.1

                proj_img, wmask, flow_bw = projective_inverse_warp(image_slice_bw, torch.squeeze( (1.0/depth_slice_bw), 3), est_poses_bw[i], data_dict['intrinsics'], format='eular')
                pose_loss_bw += image_similarity(image_slice_bw, proj_img, wmask)*0.1 #shouldnt work
                mask_loss_bw += l1loss(mask_regular, wmask)*0.1

                #Visualize first depth map
                if i==0:
                depth_slice_fw1 = depth_slice_fw
                continue

            # Get current image and depth
            depth_slice_bw = est_depths[0:-1, 0:-1, self.img_width*i:int(self.img_width), 0:-1]

            image_slice_fw = data_dict['image_seq'][0:-1, 0:-1, self.img_width*i:int(self.img_width), 0:-1]

            depth_slice_fw = torch.reshape(depth_slice_fw, (self.batch_size, self.img_height, self.img_width, 1))
            image_slice_fw = torch.reshape(image_slice_fw, (self.batch_size, self.img_height, self.img_width, 3))

            accum_pose_fw = torch.eye(4)
            accum_pose_fw = accum_pose_fw.rshape((1,4,4))
            accum_pose_fw = accum_pose_fw.repeat(self.batch_size, 1, 1)

            accum_pose_bw = torch.eye(4)
            accum_pose_bw = accum_pose_bw.rshape((1,4,4))
            accum_pose_bw = accum_pose_bw.repeat(self.batch_size, 1, 1)

            if i==4:
                depth_slice_fw5 = depth_slice_fw

            degrade=0

            # Project current view into every previous view using accumulate
            # transformation
            
            for j in range(i-1, -1, -1):
                
                previous_scene_fw = data_dict['image_seq'][0:-1, 0: -1, self.img_width * j:int(self.img_width), 0:-1]
                previous_scene_fw = previous_scene_fw.reshape(self.batch_size, self.img_height, self. img_width, 3)
                accum_pose_fw = torch.mm(pose_vec2mat(est_poses[j+1], 'euler'), accum_pose_fw) #Matrix multiplication

                # Differentiable geometric module (DGM)
                # Using depth and pose to compute warped image, and warping flow

                proj_img_fw, wmask_fw, flow_fw = projective_inverse_warp(previous_scene_fw, torch.squeeze( (1.0/depth_slice_fw), 3),accum_pose_fw,data_dict['intrinsics'],format='matrix')

                
                # Perform the same operation for backward prediction
                depth_slice_bw = est_depths_bw[0:-1, 0: -1, self.img_width * (self.num_views-j-1):int(self.img_width), 0:-1]
                depth_slice_bw = depth_slice_bw.reshape(self.batch_size, self.img_height, self. img_width, 1)
                accum_pose_bw  = torch.mm(accum_pose_bw, pose_vec2mat(est_poses_bw[self.num_views-j-1],'eular'))
                
                proj_img_bw, wmask_bw, flow_bw = projective_inverse_warp(image_slice_fw, torch.squeeze( 1.0/depth_slice_bw, dim=3), accum_pose_bw, data_dict['intrinsics'], format='matrix')
                
                
                # Compute Forward backward flow
                Fba_i, _, _ = projective_inverse_warp(-flow_bw, torch.squeeze(1.0/depth_slice_fw, 3), accum_pose_fw, data_dict['intrinsics'], format='matrix')

                Fab_i, _, _ = projective_inverse_warp(-flow_fw, torch.squeeze(1.0 / depth_slice_bw, axis=3), accum_pose_bw, data_dict['intrinsics'],format='matrix')

                #Find inconsistency
                fw_flowdiff = torch.norm(flow_fw - Fba_i, dim=3)
                bw_flowdiff = torch.norm(flow_bw - Fab_i, dim=3)

                #A Threshold to filter out moving objects or occlusion boundaries
                fw_threshold_flow = torch.where((0.05*torch.norm(flow_fw, dim=3)<=3.0), torch.ones_like(fw_flowdiff)*3.0, 0.05*torch.norm(flow_fw, dim=3)) #Recheck
                bw_threshold_flow = torch.where((0.05*torch.norm(flow_bw, dim=3)<=3.0), torch.ones_like(fw_flowdiff)*3.0, 0.05*torch.norm(flow_bw, dim=3)) #Recheck

                fw_occ_mask = torch.unsqueeze((torch.where(fw_flowdiff<=fw_threshold_flow), torch.ones_like(fw_flowdiff), torch.zeros_like(fw_flowdiff)),axis=-1)
                bw_occ_mask = torch.unsqueeze((torch.where(bw_flowdiff<=bw_threshold_flow), torch.ones_like(fw_flowdiff), torch.zeros_like(fw_flowdiff)),axis=-1)

                #A mask for valid pixels.
                wmask_fw = wmask_fw*fw_occ_mask
                wmask_bw = wmask_bw*bw_occ_mask

                # Image reprojection loss
                pose_loss_fw += image_similarity(image_slice_fw, proj_img_fw, wmask_fw)/2**degrade/10.0 #Function doesn't exist, wont work
                pose_loss_bw += image_similarity(previous_scene_fw, proj_img_bw, wmask_bw)/2**degrade/10.0 #Function doesn't exist, wont work


                # Foward back flow consistency loss
                if j == i-1:
                    consist_fw_err = l1loss(flow_fw, Fba_i, wmask_fw[:,:,:,0:2])
                    consist_loss_fw += consist_fw_err*0.01/2**degrade
                    consist_bw_err = l1loss(flow_bw, Fab_i, wmask_bw[:,:,:,0:2])
                    consist_loss_bw += consist_bw_err*0.01/2**degrade

                    # A regularization loss on mask to prevent trivial solution
                    mask_loss_fw +=l1loss(mask_regular, wmask_fw)*0.01/2**degrade
                    mask_loss_bw +=l1loss(mask_regular, wmask_bw)*0.01/2**degrade

                    flow_fw_img = flow_fw

                if i==1:
                    flow_bw_img = flow_bw

                degrade+=1

        all_losses.append(pose_loss_fw)
        all_losses.append(pose_loss_bw)
        data_dict['est_pose'] = est_poses
        data_dict['est_pose_bw'] = est_poses_bw
        
        all_losses.append(mask_loss_fw)
        all_losses.append(mask_loss_bw)
        all_losses.append(consist_loss_fw)
        all_losses.append(consist_loss_bw)
        
        total_loss = tf.reduce_mean(all_losses)

        output_dict['previous_scene_fw'] = previous_scene_fw
        output_dict['proj_img_fw'] = proj_img_fw
        output_dict['image_slice_fw'] = image_slice_fw
        output_dict['depth_slice_fw'] = depth_slice_fw

        output_dict['previous_scene_bw'] = image_slice_fw
        output_dict['proj_img_bw'] = proj_img_bw
        output_dict['image_slice_bw'] = previous_scene_fw
        output_dict['depth_slice_bw'] = depth_slice_bw

        output_dict['flow_bw'] = flow_bw_img
        output_dict['flow_fw'] = flow_fw_img

        data_dict['flow_bw'] = flow_bw
        data_dict['flow_fw'] = flow_fw

        output_dict['depth_slice_fw1'] = depth_slice_fw1
        output_dict['depth_slice_fw5'] = depth_slice_fw5

        return total_loss, all_losses, output_dict

        
    #Write Summary Funcitons
    def sub_depth(self, est_depths):
        
        depth_slice1 = est_depths[0:-1, 0:-1, self.img_width*0:int(self.img_width), 0:-1]
        depth_slice2 = est_depths[0:-1, 0:-1, self.img_width*5:int(self.img_width), 0:-1]
        depth_slice3 = est_depths[0:-1, 0:-1, self.img_width*9:int(self.img_width), 0:-1]
        depth=1.0/torch.cat((depth_slice1, depth_slice2, depth_slice3), 2)
        return depth
    
        
#     def tb_summary(self, output_dict, loss, all_losses):
        
#         color_depth_fw = torch.unsqueeze(depth_plasma(output_dict['depth_slice_fw']), 0)
        
    #Run Session
#     def save(self, sess, checkpoint_dir, step, saver):
#         '''
#         Save Checkpoints
#         '''
#         model_name = 'model'
        
#         if step == 'latest':
#             saver.save(sess,
#                        os.path.join(checkpoint_dir, model_name + '.latest'))
#         else:
#             saver.save(sess, os.path.join(checkpoint_dir, model_name), global_step=step)

            
    #Write train function
    
    
        
        